In [ ]:
# TASKS
# Get rid of CNN and NYT and use sources more clearly on the left
## Don't want any David Brooks kinda stuff mixed in there
## NYT/CNN are 'centrist' kind of

In [44]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer, TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.base import TransformerMixin, BaseEstimator
from sklearn.svm import SVC

In [21]:
bb = pd.read_csv('./jun20_bb_posts.csv', )
cnn = pd.read_csv('./jun20_cnn_posts.csv')
fox = pd.read_csv('./jun20_fox_posts.csv')
nr = pd.read_csv('./jun20_nr_posts.csv')
# nyt = pd.read_csv('./jun20_nyt_posts.csv')
vice = pd.read_csv('./jun20_vice_posts.csv')
dem = pd.read_csv('./jun21_demnow_posts.csv')
dem2 = pd.read_csv('./jun22_demnow_posts.csv')
dem3 = pd.read_csv('./jun23_demnow_posts.csv')
dem4 = pd.read_csv('./jun24_demnow_posts.csv')
inwa = pd.read_csv('./june20infowars.csv')
inwa2 = pd.read_csv('./june21infowars.csv')
inwa3 = pd.read_csv('./june22infowars.csv')
inwa4 = pd.read_csv('./june23infowars.csv')
inwa5 = pd.read_csv('./june24infowars.csv')
msnbc = pd.read_csv('./jun21_msnbc_posts.csv')
huff = pd.read_csv('./jun_24_huff_posts.csv')

In [22]:
bb['yes_right'] = 1
cnn['yes_right'] = 0
fox['yes_right'] = 1
nr['yes_right'] = 1
# nyt['yes_right'] = 0
vice['yes_right'] = 0
dem['yes_right'] = 0
dem2['yes_right'] = 0
dem3['yes_right'] = 0
dem4['yes_right'] = 0
inwa['yes_right'] = 1
inwa2['yes_right'] = 1
inwa3['yes_right'] = 1
inwa4['yes_right'] = 1
inwa5['yes_right'] = 1
msnbc['yes_right'] = 0
huff['yes_right'] = 0

In [23]:
text = pd.concat([bb, cnn, fox, nr, vice, dem, dem2, dem3, inwa, inwa2, inwa3, inwa4, msnbc, huff])

In [24]:
text.drop(['Unnamed: 0', 'category', 'urlToImage'], axis=1, inplace=True)

In [25]:
text.isna().sum()

author         4170
description      67
publishedAt       0
source            0
title             2
url               0
yes_right         0
dtype: int64

In [26]:
no_desc = text.description.isna()

In [27]:
text.dropna(subset=['description','title'], inplace=True)

In [28]:
text.isna().sum()

author         4112
description       0
publishedAt       0
source            0
title             0
url               0
yes_right         0
dtype: int64

In [29]:
text.fillna('none', inplace=True)

In [30]:
text.shape

(35129, 7)

In [31]:
len(text.url.unique())

29982

In [32]:
text.drop_duplicates('url', inplace=True)

In [33]:
text.isna().sum().sum()

0

In [34]:
text.shape

(29982, 7)

In [35]:
text.reset_index(drop=True, inplace=True)

In [36]:
text.yes_right.value_counts()

1    15098
0    14884
Name: yes_right, dtype: int64

#### Modeling

In [63]:
X = text[['description', 'title']]
# X = text['description']
y = text['yes_right']

In [64]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42, stratify=y)

# Simple CountVec

In [53]:
cvec = CountVectorizer(strip_accents='ascii', ngram_range=(1,3), stop_words='english', max_features=1250)

In [54]:
cvec.fit(X_train, y_train)

CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=1250, min_df=1,
        ngram_range=(1, 3), preprocessor=None, stop_words='english',
        strip_accents='ascii', token_pattern='(?u)\\b\\w\\w+\\b',
        tokenizer=None, vocabulary=None)

In [55]:
X_feats = pd.DataFrame(cvec.transform(X_train).todense(),
                       columns=cvec.get_feature_names())

In [56]:
word_counts = X_feats.sum(axis=0)
word_counts.sort_values(ascending = False).head(20)

trump                     3758
president                 3221
new                       1979
news                      1956
fox                       1581
said                      1530
donald                    1400
fox news                  1362
donald trump              1277
house                     1156
president donald          1139
president donald trump    1058
north                      961
president trump            931
people                     917
says                       872
state                      860
year                       838
cnn                        810
white                      784
dtype: int64

In [57]:
cv_train = cvec.transform(X_train).todense()
cv_test = cvec.transform(X_test).todense()

In [58]:
lr = LogisticRegression(random_state=42)
lr.fit(cv_train, y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=42, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [59]:
lr.score(cv_train, y_train)

0.76452014586854045

In [60]:
lr.score(cv_test, y_test)

0.72945570971184637

In [61]:
cfs = lr.coef_[0]
fts = cvec.get_feature_names()

In [62]:
pd.DataFrame(
    list(zip(fts, np.abs(cfs), cfs)),
    columns=['feat','abs','coef']).sort_values('abs',ascending=False).head(25)

,feat,abs,coef
746,national review,2.903722,2.903722
513,huffpost,2.760954,-2.760954
216,cnn,2.334393,-2.334393
824,photographer,2.311054,-2.311054
936,reuters,2.294384,2.294384
545,insight,2.250141,2.250141
899,reacts,2.034375,2.034375
379,fast facts,2.001188,-2.001188
154,broadcast,1.938742,1.938742
73,amp,1.892118,1.892118


# Pipe

In [65]:
class DfExtract(BaseEstimator, TransformerMixin):
    def __init__(self, column):
        self.column = column
    def fit(self, X, y=None):
        return self
    def transform(self, X, y=None):
        if len(self.column) > 1:
            return pd.DataFrame(X[self.column])
        else:
            return pd.Series(X[self.column[0]]) 

In [66]:
pipe = Pipeline([
    ('features', FeatureUnion([
        ('tit', Pipeline([
            ('ext', DfExtract(['title'])),
#             ('cvec', CountVectorizer(ngram_range=(1,2), stop_words='english')),
            ('tf', TfidfVectorizer(stop_words='english', ngram_range=(1,2))) #was transformer
        ])),
        ('desc', Pipeline([
            ('ext', DfExtract(['description'])),
#             ('cvec', CountVectorizer(ngram_range=(1,2), stop_words='english')),
            ('tf', TfidfVectorizer(stop_words='english', ngram_range=(1,2))) # was transformer
        ]))
    ])),
#     ('lr', LogisticRegression(random_state=42))
    ('svc', SVC(random_state=42))
])

In [67]:
params = {
#     'cvec__max_df': (0.5, 0.75, 1.0),
#     'tf__norm': ['l1', 'l2', None],
#     'lr__penalty': ['l1','l2'],
#     'lr__C': [0.75, 1.0]
#     'rf__n_estimators': [4,10],
#     'rf__max_depth': [None, 5],
#     'rf__warm_start': [True, False],
    'svc__kernel': ['rbf', 'sigmoid', 'poly']
}

gs = GridSearchCV(pipe, params, scoring='roc_auc')

In [ ]:
gs.fit(X_train, y_train)

In [43]:
gs.score(X_test, y_test)

0.87140087315415826

In [28]:
gs.best_params_

{'lr__C': 1.0, 'lr__penalty': 'l2'}

In [29]:
coefs = list(gs.best_estimator_.steps[1][1].coef_[0])

In [30]:
len(coefs)

378884

In [50]:
# gs.best_estimator_.steps[0][1].get_feature_names()

In [57]:
# lr_coefs = pd.DataFrame(
#     list(
#         zip(X_test.columns, np.abs(
#             coefs))), columns=['feature','coef_abs'])
# lr_coefs.sort_values('coef_abs', ascending=False, inplace=True)

In [31]:
def manual_test():
    title = input('Article title: \n')
    desc = input('Article description: \n')
    df = [{'title': '{}'.format(title),
                'description': '{}'.format(desc)
                }]
    df=pd.DataFrame(df)
    pred = gs.predict(df)
    if pred == 1:
        print("Result: Right wing")
    else:
        print("Result: Not right wing")

In [32]:
manual_test()

Article title: 
Democrats are a bunch of whiny snowflakes
Article description: 
When will they grow up? 
Result: Right wing


In [33]:
manual_test()

Article title: 
Trump's foreign policy is a disaster
Article description: 
How the global economy will be affected by the Trump administration's trade wars
Result: Right wing
